In [1]:
import pandas as pd
import numpy as np
import joblib

In [ ]:
DATA_PATH = "../data/processed/engineered_coffee_health.csv"
df = pd.read_csv(DATA_PATH)

MODEL_PATH = "../outputs/models/final_health_risk_model.pkl"
model = joblib.load(MODEL_PATH)

SCALER_PATH = "../outputs/models/scaler.pkl"
scaler = joblib.load(SCALER_PATH)
expected_features = list(getattr(scaler, 'feature_names_in_', []))

In [ ]:
def predict_health_risk(user_features: dict):
    """
    Predicts health risk based on user features.
    user_features: dict of feature_name -> value
    returns: predicted risk level as string
    """
    X = pd.DataFrame([user_features])

    if 'BMI' in X:
        X['BMI_Category'] = pd.cut(X['BMI'], bins=[0,18.5,24.9,29.9,100], labels=['Underweight','Normal','Overweight','Obese'])
        X['BMI_Category'] = X['BMI_Category'].astype('category').cat.codes

    if expected_features:
        for col in expected_features:
            if col not in X.columns:
                X[col] = 0
        X = X[expected_features]

    if scaler:
        X_scaled = scaler.transform(X)
    else:
        X_scaled = X.values
    
    risk_pred = model.predict(X_scaled)[0]
    return risk_pred


In [ ]:
def generate_recommendations(user_features: dict):
    """
    Generates personalized lifestyle recommendations based on predicted health risk.
    """
    risk = predict_health_risk(user_features)
    recommendations = []

    if risk == "None":
        recommendations.append("Maintain current healthy habits.")
        recommendations.append("Keep balanced diet, moderate coffee intake, and regular exercise.")
    elif risk == "Mild":
        recommendations.append("Reduce coffee intake and monitor stress levels.")
        recommendations.append("Aim for at least 7 hours of sleep daily.")
        recommendations.append("Increase light physical activity, like walking or yoga.")
    elif risk == "Moderate":
        recommendations.append("Consult a healthcare professional for personalized advice.")
        recommendations.append("Follow a structured diet and exercise plan.")
        recommendations.append("Avoid smoking and limit alcohol consumption.")
        recommendations.append("Regularly monitor BMI, heart rate, and sleep quality.")

    # Feature-specific advice
    if user_features.get("Sleep_Hours", 8) < 6:
        recommendations.append("Increase sleep to 7–8 hours for better health.")
    if user_features.get("Coffee_Intake", 0) > 3:
        recommendations.append("Consider reducing coffee intake to 1–2 cups daily.")
    if user_features.get("Physical_Activity_Hours", 0) < 1:
        recommendations.append("Try to include at least 30 minutes of daily physical activity.")
    if user_features.get("BMI", 22) > 25:
        recommendations.append("Maintain a healthy weight through diet and exercise.")

    return risk, recommendations

In [5]:
sample_users = [
    {"Age": 25, "Gender": 0, "Coffee_Intake": 1, "Sleep_Hours": 7, "BMI": 22,
     "Physical_Activity_Hours": 2, "Stress_Level": 2, "Alcohol_Consumption": 1, "Smoking": 0},
    
    {"Age": 40, "Gender": 1, "Coffee_Intake": 4, "Sleep_Hours": 5, "BMI": 27,
     "Physical_Activity_Hours": 0.5, "Stress_Level": 4, "Alcohol_Consumption": 3, "Smoking": 1},
    
    {"Age": 30, "Gender": 0, "Coffee_Intake": 2, "Sleep_Hours": 6, "BMI": 24,
     "Physical_Activity_Hours": 1, "Stress_Level": 3, "Alcohol_Consumption": 2, "Smoking": 0}
]

for i, user in enumerate(sample_users, 1):
    risk, recs = generate_recommendations(user)
    print(f"\n--- Sample User {i} ---")
    print("Predicted Health Risk:", risk)
    print("Recommendations:")
    for r in recs:
        print("-", r)


--- Sample User 1 ---
Predicted Health Risk: Mild
Recommendations:
- Reduce coffee intake and monitor stress levels.
- Aim for at least 7 hours of sleep daily.
- Increase light physical activity, like walking or yoga.

--- Sample User 2 ---
Predicted Health Risk: Mild
Recommendations:
- Reduce coffee intake and monitor stress levels.
- Aim for at least 7 hours of sleep daily.
- Increase light physical activity, like walking or yoga.
- Increase sleep to 7–8 hours for better health.
- Consider reducing coffee intake to 1–2 cups daily.
- Try to include at least 30 minutes of daily physical activity.
- Maintain a healthy weight through diet and exercise.

--- Sample User 3 ---
Predicted Health Risk: Mild
Recommendations:
- Reduce coffee intake and monitor stress levels.
- Aim for at least 7 hours of sleep daily.
- Increase light physical activity, like walking or yoga.


In [6]:
recommendations_list = []
for user in sample_users:
    risk, recs = generate_recommendations(user)
    recommendations_list.append({
        "User_Features": user,
        "Predicted_Risk": risk,
        "Recommendations": recs
    })

df_recommendations = pd.DataFrame(recommendations_list)
df_recommendations.to_csv("../outputs/results/recommendations.csv", index=False)
